In [3]:
import sys
sys.path.append('..')

In [4]:
from VisionSystem import VisionSystem, VideoStream, VisualObject
from VisionSystem.DetectionModel import ThreshBlob
from DisplayPane import DisplayPane
from KickerSystem import KickerSystem
from DriveSystem import DriveSystem

before {}
after {13: ('/home/pi/SoccerRobot/DriveSystem/DriveSystem.py, line 10', <gpiozero.LED object on pin GPIO13, active_high=True, is_active=False>), 26: ('/home/pi/SoccerRobot/DriveSystem/DriveSystem.py, line 12', <gpiozero.LED object on pin GPIO26, active_high=True, is_active=False>)}
before {13: ('/home/pi/SoccerRobot/DriveSystem/DriveSystem.py, line 10', <gpiozero.LED object on pin GPIO13, active_high=True, is_active=False>), 26: ('/home/pi/SoccerRobot/DriveSystem/DriveSystem.py, line 12', <gpiozero.LED object on pin GPIO26, active_high=True, is_active=False>)}
after {13: ('/home/pi/SoccerRobot/DriveSystem/DriveSystem.py, line 10', <gpiozero.LED object on pin GPIO13, active_high=True, is_active=False>), 26: ('/home/pi/SoccerRobot/DriveSystem/DriveSystem.py, line 12', <gpiozero.LED object on pin GPIO26, active_high=True, is_active=False>), 5: ('/home/pi/SoccerRobot/DriveSystem/DriveSystem.py, line 10', <gpiozero.LED object on pin GPIO5, active_high=True, is_active=False>), 12: 

In [5]:
drive = DriveSystem()
kicker = KickerSystem()
kicker.setup() # dont ask

In [6]:
def load_or_create_new_threshblob(path):
    try:
        model = ThreshBlob.load(path)
        print("Loaded " + path)
    except Exception:
        model = ThreshBlob()
    finally:
        return model
    

def setup_vision_system(resolution):
    objects_to_size_and_result_limit = [
        ("ball", (0.043, 0.043, 0.043), 1),
        ("obstacle", (0.18, 0.18, 0.2), None),
        # 30 centimetres long, 10 cm high? i guess
        ("blue_goal", (0, 0, 0.1), 1),
        ("yellow_goal", (0, 0, 0.1), 1)
    ]

    return VisionSystem(
        resolution=resolution,
        objects_to_track={
            name: VisualObject(
                real_size=size,
                detection_model=load_or_create_new_threshblob(f"../models/{name}.threshblob.pkl"),
                result_limit=result_limit
            ) for name, size, result_limit in objects_to_size_and_result_limit
        }
    )

video = VideoStream(downsample_scale=5, crop=((0.13, 0), (.9, 1)))
vision = setup_vision_system(video.resolution)

PiCameraMMALError: Failed to enable connection: Out of resources

In [5]:
from DisplayPane.Interactor.VisionSystemTuner import VisionSystemTuner

DisplayPane(vision_system=vision, video_stream=video, interactors=[VisionSystemTuner(vision)])

DisplayPane(children=(HBox(children=(VBox(children=(Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'r…

In [6]:
# RUN THIS CELL TO SAVE THE MODELs tinkered with
for name, obj in vision.objects_to_track.items():
    obj.detection_model.save(f"../models/{name}.threshblob.pkl")

In [7]:
def get_vision_results_vrep_format():
    objs = vision.objects_to_track # for shorthand

    def vrep_format(bearings_distances, multi=False):
        if any(bearings_distances):
            if multi:
                bds = bearings_distances[::-1]
                for idx, (bear, dist) in enumerate(bds):
                    bds[idx] = (np.pi - bear), dist
                return bds
            else:
                bear, dist = bearings_distances[0][::-1]
                return (np.pi - bear), dist
        else:
            return None

    return (
        vrep_format(objs["ball"].bearings_distances),
        vrep_format(objs["blue_goal"].bearings_distances),
        vrep_format(objs["yellow_goal"].bearings_distances),
        vrep_format(objs["obstacle"].bearings_distances, multi=True),
    )

In [8]:
from math import *
from time import sleep

T_SPEED = 0.03
R_SPEED = 0.02

for frame in video:
    vision.update_with_frame(frame)
    
    ball_br, blue_br, yellow_br, obstacle_brs = get_vision_results_vrep_format()
    
    if ball_br is not None:
        ball_bear, ball_range = (-pi / 2, 1)
        print("BALL", ball_br)
        t_speed = T_SPEED if ball_range > 0.3 else ball_range / 0.3 * T_SPEED
        r_speed = R_SPEED if ball_bear > 0.5 else ball_bear / 0.5 * R_SPEED
        
        drive.set_desired_motion(
            t_speed * cos(ball_bear),
            t_speed * sin(ball_bear),
            r_speed * ball_bear)
        
        sleep(0.2)
        
        drive.set_desired_motion(0,0,0)
        sleep(0.1)
        

../VisionSystem/DetectionModel/DetectionResult.py:29: RuntimeWarning: overflow encountered in long_scalars
  return 0.5*np.abs(np.dot(x, np.roll(y, 1))-np.dot(y, np.roll(x, 1)))


BALL (0.9908634040343494, -0.11252317261277611)
set desired motion 1.8369701987210296e-18 -0.03 0.09869604401089359
driving left, right, back -0.48270336656492074 0.9268355646139419 0.22206609902451066
set desired motion 0 0 0
driving left, right, back 0.0 0.0 0.0
BALL (0.8926288835269626, 0.5210551891720965)
set desired motion 1.8369701987210296e-18 -0.03 0.09869604401089359
driving left, right, back -0.48270336656492074 0.9268355646139419 0.22206609902451066


../VisionSystem/DetectionModel/ThreshBlob/ThreshBlob.py:22: RuntimeWarning: overflow encountered in exp
  return (0.1407675 * np.exp(0.85425173 * img_dist / RESCALE / frac_of_full_res) + 0.79365708) * DIST_PER_POINT


set desired motion 0 0 0
driving left, right, back 0.0 0.0 0.0
BALL (0.6968057876920698, 1.1769977129200961)
set desired motion 1.8369701987210296e-18 -0.03 0.09869604401089359
driving left, right, back -0.48270336656492074 0.9268355646139419 0.22206609902451066
set desired motion 0 0 0
driving left, right, back 0.0 0.0 0.0
BALL (0.26350624288284547, 0.162761793261713)
set desired motion 1.8369701987210296e-18 -0.03 0.09869604401089359
driving left, right, back -0.48270336656492074 0.9268355646139419 0.22206609902451066
set desired motion 0 0 0
driving left, right, back 0.0 0.0 0.0
BALL (0.28823267147970033, 0.4740445139621182)
set desired motion 1.8369701987210296e-18 -0.03 0.09869604401089359
driving left, right, back -0.48270336656492074 0.9268355646139419 0.22206609902451066
set desired motion 0 0 0
driving left, right, back 0.0 0.0 0.0
BALL (0.27719261861935823, 0.8515497279175493)
set desired motion 1.8369701987210296e-18 -0.03 0.09869604401089359
driving left, right, back -0.482

KeyboardInterrupt: 

In [9]:
drive.set_desired_motion(0,0,0)

set desired motion 0 0 0
driving left, right, back 0.0 0.0 0.0


In [ ]:
# run the kicker
